In [1]:
import requests
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import json
import time
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Legacy Graph URL (not used in the current code)
# The Graph endpoint for Uniswap V3 on Arbitrum
# GRAPH_URL = "https://gateway-arbitrum.network.thegraph.com/api/418eadaa2d81cca124449b86392c8a70/subgraphs/id/FbCGRftH4a3yZugY7TnbYgPJVEv2LvMT6oF1fxPe9aJM"

# def query_graph(query: str, variables: Dict = None) -> Dict:
#     """
#     Query The Graph API
#     """
#     payload = {
#         "query": query,
#         "variables": variables or {}
#     }
    
#     try:
#         response = requests.post(GRAPH_URL, json=payload, timeout=30)
#         response.raise_for_status()
#         data = response.json()
        
#         if "errors" in data:
#             print(f"GraphQL errors: {data['errors']}")
#             return {}
            
#         return data.get("data", {})
#     except Exception as e:
#         print(f"Error querying The Graph: {e}")
#         return {}

In [3]:
# The proxy endpoint for the poolfish.xyz website
PROXY_URL = "https://poolfish.xyz/api/subgraph"
CHAIN_ID = 42161  # Arbitrum One
DEX_KEY = "uniswap"

HEADERS = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    'dnt': '1',
    'origin': 'https://poolfish.xyz',
    'priority': 'u=1, i',
    'referer': 'https://poolfish.xyz/calculators/uniswap?token0=0x82af49447d8a07e3bd95bd0d56f35241523fbab1&token1=0xaf88d065e77c8cc2239327c5edb3a432268e5831&feeTier=500',
    'sec-ch-ua': '"Chromium";v="141", "Not?A_Brand";v="8"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'
}

def query_graph(query: str) -> Dict:
    """
    Query the Poolfish proxy endpoint (which in turn queries The Graph)
    """
    payload = {
        "query": query,
        "dexKey": DEX_KEY,
        "chainId": CHAIN_ID
    }
    
    time.sleep(0.5)
    
    try:
        response = requests.post(PROXY_URL, json=payload, headers=HEADERS, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if "errors" in data:
            print(f"GraphQL errors: {data['errors']}")
            return {}
            
        return data.get("data", {})
    except Exception as e:
        print(f"Error querying the proxy: {e}")
        # Print response text if it's not a successful 2xx code
        if 'response' in locals():
            print(f"Response Status: {response.status_code}")
            print(f"Response Text: {response.text}")
        return {}

In [ ]:
def get_pool_day_data(pool_address: str, num_days: int = 7) -> List[Dict]:
    """
    Get historical pool day data for volume calculation
    """
    query = f"""
    {{
        poolDayDatas(first: {num_days}, orderBy: date, orderDirection: desc, where: {{pool: "{pool_address.lower()}"}}) {{
            date
            volumeUSD
            tvlUSD
            feesUSD
            open
            high
            low
            close
        }}
    }}
    """
    
    result = query_graph(query)
    return result.get("poolDayDatas", [])

def get_pool_ticks(pool_address: str, skip: int = 0, first: int = 1000) -> List[Dict]:
    """
    Get pool ticks data for liquidity calculation
    """
    query = f"""
    {{
        ticks(first: {first}, skip: {skip}, where: {{poolAddress: "{pool_address.lower()}"}}, orderBy: tickIdx) {{
            tickIdx
            liquidityNet
            price0
            price1
        }}
    }}
    """
    
    result = query_graph(query)
    return result.get("ticks", [])

def get_all_pool_ticks(pool_address: str) -> List[Dict]:
    """
    Get all ticks for a pool (paginated)
    """
    all_ticks = []
    skip = 0
    batch_size = 1000
    
    while True:
        ticks = get_pool_ticks(pool_address, skip, batch_size)
        if not ticks:
            break
            
        all_ticks.extend(ticks)
        
        if len(ticks) < batch_size:
            break
            
        skip += batch_size
        time.sleep(0.1)  # Rate limiting
    
    return all_ticks

def get_token_data(token_address: str) -> Dict:
    """
    Get token data including decimals and current price from the Graph API
    """
    query = f"""
    {{
        token(id: "{token_address.lower()}") {{
            id
            name
            symbol
            decimals
            volumeUSD
            totalValueLockedUSD
            tokenDayData(first: 1, orderBy: date, orderDirection: desc) {{
                priceUSD
                date
            }}
        }}
    }}
    """
    
    result = query_graph(query)
    token = result.get("token", {})
    
    # Add priceUSD from the most recent tokenDayData
    if token and 'tokenDayData' in token and token['tokenDayData']:
        token['priceUSD'] = token['tokenDayData'][0]['priceUSD']
        del token['tokenDayData']  # Remove the raw data since we extracted what we need
        
    return token

def sort_tokens(token0_addr: str, token1_addr: str) -> List[str]:
    """
    Sort tokens by address (same as sortTokens in uniswap.ts)
    """
    if token0_addr.lower() < token1_addr.lower():
        return [token0_addr, token1_addr]
    return [token1_addr, token0_addr]

def get_pool_from_pair(token0_addr: str, token1_addr: str) -> List[Dict]:
    """
    Get pool data for a token pair (similar to getPoolFromPair in uniswap.ts)
    """
    sorted_tokens = sort_tokens(token0_addr, token1_addr)
    
    query = f"""
    {{
        pools(orderBy: feeTier, where: {{
            token0: "{sorted_tokens[0].lower()}",
            token1: "{sorted_tokens[1].lower()}"
        }}) {{
            id
            feeTier
            token0 {{
                id
                symbol
            }}
            token1 {{
                id
                symbol
            }}
        }}
    }}
    """
    
    result = query_graph(query)
    return result.get("pools", [])


In [5]:
# pool_addr = "0x5969efdde3cf5c0d9a88ae51e47d721096a97203"
# token0 = "0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f"  # WBTC
# token1 = "0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9"  # USDT
pool_addr = "0xc6962004f452be9203591991d15f6b388e09e8d0"
token0 = "0x82af49447d8a07e3bd95bd0d56f35241523fbab1"  # WETH
token1 = "0xaf88d065e77c8cc2239327c5edb3a432268e5831"  # USDC

# Validate pool address
pools = get_pool_from_pair(token0, token1)
if not pools:
    raise ValueError(f"No pools found for token pair {token0} and {token1}")

# Find the pool with matching address
pool_found = False
for pool in pools:
    if pool['id'].lower() == pool_addr.lower():
        print(f"✅ Valid pool found:")
        print(f"Pool ID: {pool['id']}")
        print(f"Fee Tier: {int(pool['feeTier'])/1_000_000:.3%}")
        print(f"Token0: {pool['token0']['symbol']}")
        print(f"Token1: {pool['token1']['symbol']}")
        pool_found = True
        break

if not pool_found:
    available_pools = "\n".join([f"- {p['id']} (Fee: {int(p['feeTier'])/1_000_000:.3%})" for p in pools])
    raise ValueError(f"Pool address {pool_addr} not found for this token pair.\nAvailable pools:\n{available_pools}")

# Get pool data
print("\nFetching pool data...")
pool_day_data = get_pool_day_data(pool_addr, 90)
pool_ticks_data = get_all_pool_ticks(pool_addr)

# Get token data
print("\nFetching token data...")
token0_data = get_token_data(token0)
token1_data = get_token_data(token1)

print(f"Token0 ({token0_data['symbol']}):")
print(f"  - Decimals: {token0_data['decimals']}")
print(f"  - Price: ${float(token0_data.get('priceUSD', 0)):.2f}")
print(f"\nToken1 ({token1_data['symbol']}):")
print(f"  - Decimals: {token1_data['decimals']}")
print(f"  - Price: ${float(token1_data.get('priceUSD', 0)):.2f}")

# Save pool data
print("\nSaving data...")
df = pd.DataFrame(pool_day_data)
df.to_csv(f'{pool_addr}_pool_day_data.csv', index=False)

df = pd.DataFrame(pool_ticks_data)
df.to_csv(f'{pool_addr}_pool_ticks_data.csv', index=False)

# Save token data
token_data = {
    'token0': token0_data,
    'token1': token1_data,
    'pool': {
        'id': pool_addr,
        'feeTier': next(p['feeTier'] for p in pools if p['id'].lower() == pool_addr.lower())
    }
}
with open(f'{pool_addr}_token_data.json', 'w') as f:
    json.dump(token_data, f, indent=2)

print("✅ All data saved successfully!")

✅ Valid pool found:
Pool ID: 0xc6962004f452be9203591991d15f6b388e09e8d0
Fee Tier: 0.050%
Token0: WETH
Token1: USDC

Fetching pool data...

Fetching token data...
Token0 (WETH):
  - Decimals: 18
  - Price: $3853.49

Token1 (USDC):
  - Decimals: 6
  - Price: $1.00

Saving data...
✅ All data saved successfully!
